In [1]:
from gazpacho import get, Soup

In [2]:
url = 'https://www.capfriendly.com/'
html = get(url)

In [3]:
soup = Soup(html)

In [4]:
table = soup.find('table', {'id': 'ich'})

In [5]:
str(table)[:100]

'<table id="ich" class="sortablex tblcf index tbl sortable"><thead><tr class="column_head"><th class='

In [6]:
trs = table.find('tr', {'class': 'tmx'})

In [7]:
tr = trs[0]

In [8]:
tr.find('a', mode='first').text

'Arizona Coyotes'

In [9]:
tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text

'$85,187,244'

In [10]:
def parse_tr(tr):
    team = tr.find('a', mode='first').text
    cap = tr.find('td', {'data-label': 'PROJECTED CAP HIT'}, strict=True).text
    cap = float(cap.replace(',', '').replace('$', ''))
    return team, cap

In [11]:
cap_hits = [parse_tr(tr) for tr in trs]

In [12]:
url = 'https://www.hockey-reference.com/friv/playoff_prob.fcgi'
html = get(url)

In [13]:
soup = Soup(html)

In [14]:
import pandas as pd

In [15]:
east = pd.read_html(str(soup.find('table')[0]))[0]
west = pd.read_html(str(soup.find('table')[1]))[0]

In [16]:
df = pd.concat([east, west])[['Team', 'W']].reset_index(drop=True)
df['W'] = df['W'].apply(pd.to_numeric, errors='coerce')
wins = df.dropna()

In [17]:
cap_hits = pd.DataFrame(cap_hits, columns=['Team', 'spend'])
df = pd.merge(wins, cap_hits, on='Team', how='left')
df['mpw'] = round(df['spend'] / df['W'] / 1_000_000, 2)
df.sort_values('mpw', ascending=True)

,Team,W,spend,mpw
16,Colorado Avalanche,49.9,76107413.0,1.53
1,Tampa Bay Lightning,50.6,80238349.0,1.59
0,Boston Bruins,50.6,81770015.0,1.62
8,Washington Capitals,49.4,81564505.0,1.65
17,St. Louis Blues,48.3,83216941.0,1.72
10,Pittsburgh Penguins,47.0,81786224.0,1.74
9,Philadelphia Flyers,47.2,82499144.0,1.75
18,Dallas Stars,45.6,82023704.0,1.80
11,New York Islanders,43.5,78223911.0,1.80
14,New York Rangers,42.9,77609652.0,1.81


In [18]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/oOX5qIDkzDjeo/giphy.gif">')

### Saving results

In [19]:
df.to_csv('../data/mpw.csv', index=False)

In [20]:
df.head()

,Team,W,spend,mpw
0,Boston Bruins,50.6,81770015.0,1.62
1,Tampa Bay Lightning,50.6,80238349.0,1.59
2,Toronto Maple Leafs,43.0,95312937.0,2.22
3,Florida Panthers,41.1,81302145.0,1.98
4,Montreal Canadiens,37.9,77166188.0,2.04


In [21]:
df['date_fetched'] = pd.Timestamp('today')

In [22]:
df.head()

,Team,W,spend,mpw,date_fetched
0,Boston Bruins,50.6,81770015.0,1.62,2020-03-05 14:18:31.008046
1,Tampa Bay Lightning,50.6,80238349.0,1.59,2020-03-05 14:18:31.008046
2,Toronto Maple Leafs,43.0,95312937.0,2.22,2020-03-05 14:18:31.008046
3,Florida Panthers,41.1,81302145.0,1.98,2020-03-05 14:18:31.008046
4,Montreal Canadiens,37.9,77166188.0,2.04,2020-03-05 14:18:31.008046


In [23]:
import sqlite3

con = sqlite3.connect('../data/mpw.db')

df.to_sql('teams', con, index=False, if_exists='append')

In [24]:
pd.read_sql('''
    select 
    * 
    from teams 
    where mpw > 2 and W < 41
    order by mpw desc
''', con)

,Team,W,spend,mpw,date_fetched
0,Detroit Red Wings,18.9,79968736.0,4.23,2020-03-05 14:18:15.070075
1,Detroit Red Wings,18.9,79968736.0,4.23,2020-03-05 14:18:31.008046
2,Detroit Red Wings,24.7,80103733.0,3.24,2020-01-02 00:17:50.488401
3,Ottawa Senators,29.1,74410642.0,2.56,2020-03-05 14:18:15.070075
4,Ottawa Senators,29.1,74410642.0,2.56,2020-03-05 14:18:31.008046
5,Anaheim Ducks,33.3,79240195.0,2.38,2020-03-05 14:18:15.070075
6,Anaheim Ducks,33.3,79240195.0,2.38,2020-03-05 14:18:31.008046
7,San Jose Sharks,34.7,80797623.0,2.33,2020-01-02 00:17:50.488401
8,Los Angeles Kings,32.3,74508673.0,2.31,2020-03-05 14:18:15.070075
9,Los Angeles Kings,32.3,74508673.0,2.31,2020-03-05 14:18:31.008046
